In [1]:
import pandas as pd
# https://www.avera.org/patients-visitors/price-transparency/

In [2]:
import csv
import subprocess
from urllib.parse import urlparse
import os

from dateutil.parser import parse as parse_datetime
import pandas as pd

In [3]:
TARGET_COLUMNS = [ 
    'hospital_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'hcpcs_cpt',
    'modifiers',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_units',
    'billing_class',
    'setting',
    'payer_category',
    'payer',
    'plan',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_payer_notes'
]

In [4]:
ccn = "161321"
url = "https://www.avera.org/app/files/public/887b57a5-04fc-4a68-9636-d6f0d9b2dfab/460224743_avera-merrill-pioneer_standardcharges.xlsx"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [5]:
import httpx
import os

def create_session():
    session = httpx.Client(http2=True)
    
    session.headers = {
        'authority': 'www.avera.org',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }
    
    resp = session.get("https://www.avera.org/patients-visitors/price-transparency/")
    print(resp.url)
    
    return session

def download_file(session, url, filename):
    if os.path.isfile(filename):
        print("{} already downloaded - skipping".format(filename))
        return
    
    resp = session.get(url)
    print(resp.url)
    
    out_f = open(filename, "wb")
    out_f.write(resp.content)
    out_f.close()
    
session = create_session()
download_file(session, url, filename)

https://www.avera.org/patients-visitors/price-transparency/
460224743_avera-merrill-pioneer_standardcharges.xlsx already downloaded - skipping


In [ ]:
df_in = pd.read_excel(filename, skiprows=1, sheet_name='CDM')
df_in

In [ ]:
df_mid = pd.DataFrame(df_in)
df_mid = df_mid.rename(columns={
    'CPT': 'cpt',
    'Charge Code': 'local_code',
    'Description': 'description',
    'Hcpcs': 'hcpcs',
    'Revcode': 'rev_code'
})

columns = df_mid.columns.to_list()
columns.pop(columns.index('Gross Charge'))
df_mid = df_mid[columns + ['Gross Charge']]

del df_mid['Facility Name']

df_mid

In [ ]:
money_columns = df_mid.columns[5:]
remaining_columns = df_mid.columns.to_list()[:5]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer', value_name='standard_charge')
df_mid

In [ ]:
df_mid['hcpcs_cpt'] = df_mid.apply(lambda row: row['cpt'] if row['cpt'] is not None else row['hcpcs'], axis=1)
del df_mid['cpt']
del df_mid['hcpcs']
df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].fillna('')
df_mid

In [ ]:
def payer_category_from_payer(payer):
    if payer == 'Gross Charge':
        return 'gross'
    elif payer == 'Discounted Cash Price':
        return 'cash'
    elif payer == 'Minimum Negotiated Rate':
        return 'min'
    elif payer == 'Maximum Negotiated Rate':
        return 'max'
    
    return 'payer'

df_mid['payer_category'] = df_mid['payer'].apply(payer_category_from_payer)
df_mid

In [ ]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
df_mid['rev_code'] = df_mid['rev_code'].fillna('')
df_mid['rev_code'] = df_mid['rev_code'][:4] # 0964an
df_mid['local_code'] = df_mid['local_code'].fillna('')
df_mid['code'] = ''
df_mid['ms_drg'] = ''
df_mid['apr_drg'] = ''
df_mid['apr_drg'] = ''
df_mid['modifiers'] = ''
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['ndc'] = ''
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_units'] = None
df_mid['billing_class'] = 1
df_mid['setting'] = 1
df_mid['plan'] = ''
df_mid['standard_charge_percent'] = None
df_mid['contracting_method'] = None
df_mid['additional_payer_notes'] = None
df_mid

In [ ]:
df_mid = df_mid.dropna(subset=['standard_charge'])
df_mid

In [ ]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
del df_out['apc']
df_out

In [ ]:
df_out.to_csv(ccn + ".csv", index=False, quoting=csv.QUOTE_ALL)